# 🗃️ Database Models and Schema Design

Welcome to the world of data modeling! In this notebook, you'll learn how to define the structure of your data both in the database and in your API. 📊

By the end of this guide, you'll have:
- ✅ SQLAlchemy models that define your database tables
- ✅ Pydantic schemas that validate your API data
- ✅ A clear understanding of how database and API models work together
- ✅ The foundation for storing and managing todo items

Let's design the blueprint for our data! 🎯

# 🤔 Understanding Models vs Schemas

Before we dive in, let's understand the difference between **models** and **schemas** - this is crucial!

## Real-World Analogy: Building a House 🏠

Think of building a house with both **architectural plans** and **delivery specifications**:

### **Database Models = House Foundation & Structure**
- 🏗️ **Defines how data is stored** in the database
- 🔧 **Handles database operations** (create tables, relationships, constraints)
- 🗄️ **Lives in the database layer**
- 📋 **Example**: "The todos table has id, name, and completed columns"

### **API Schemas = Delivery Truck Specifications**
- 📦 **Defines how data travels** between API and clients
- ✅ **Validates incoming/outgoing data**
- 🌐 **Lives in the API layer**
- 📋 **Example**: "API requests must have name (string) and completed (boolean)"

## Why Do We Need Both? 🤷‍♀️

```
🌐 Frontend (React)
    ↕ JSON Data (validated by Schemas)
🚀 FastAPI Backend
    ↕ Python Objects (converted between Schemas and Models)
🗄️ PostgreSQL Database (structured by Models)
```

### **Without Models and Schemas:**
```python
# 😱 Nightmare scenario - no structure!
@app.post("/todos")
def create_todo(data: dict):  # Any dict? What fields?
    # How do we validate this?
    # How do we save it to database?
    # What if someone sends {"hacker": "payload"}?
```

### **With Models and Schemas:**
```python
# 🎉 Clean, safe, predictable!
@app.post("/todos")
def create_todo(todo: schemas.TodoRequest):  # Validated input
    db_todo = models.Todo(**todo.dict())     # Safe database object
    # Everything is validated and type-safe!
```

## The Two-Layer System 🏗️

| Aspect | Database Models | API Schemas |
|--------|-----------------|-------------|
| **Purpose** | Store data | Validate data |
| **Location** | `models.py` | `schemas.py` |
| **Technology** | SQLAlchemy | Pydantic |
| **Focus** | Database structure | API interface |
| **Example** | `class Todo(Base)` | `class TodoRequest(BaseModel)` |

# 🔗 Database Connection Setup

Before we create models, let's set up our database connection system.

## Understanding database.py 📄

Let's examine our database connection file:

In [ ]:
# Make sure we're in the backend directory
%cd 001-fastapi-backend

In [ ]:
# Let's examine our database configuration
!cat database.py

## Breaking Down database.py 🔍

Let's understand each component:

### 1. **Imports and Environment Loading**
```python
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv

load_dotenv()
```
- **SQLAlchemy imports**: Core database toolkit components
- **Environment loading**: Gets our database credentials from .env

### 2. **Connection String Building**
```python
user = os.environ['DATABASE_USER']
password = os.environ['DATABASE_PASSWORD']
host = os.environ['DATABASE_HOST']
port = os.environ['DATABASE_PORT']
db_name = os.environ['DATABASE_NAME']

SQLALCHEMY_DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{db_name}"
```
This dynamically builds our PostgreSQL connection string from environment variables.

### 3. **Database Engine**
```python
engine = create_engine(SQLALCHEMY_DATABASE_URL)
```
The **engine** is like the "motor" that powers all database operations.

### 4. **Session Factory**
```python
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
```
Creates a **factory** for database sessions (connections).
- **`autocommit=False`**: We control when changes are saved
- **`autoflush=False`**: We control when changes are sent to database

### 5. **Base Class**
```python
Base = declarative_base()
```
This is the **parent class** that all our database models will inherit from.

## Testing Database Connection 🧪

In [ ]:
# Test our database connection setup
try:
    from database import engine, SessionLocal, Base
    
    print("🔗 Database Connection Test:")
    
    # Test engine creation
    print(f"✅ Engine created: {engine}")
    
    # Test session creation
    session = SessionLocal()
    print(f"✅ Session created: {session}")
    
    # Test connection
    connection = engine.connect()
    print("✅ Database connection successful!")
    
    # Clean up
    connection.close()
    session.close()
    
    print(f"✅ Base class ready: {Base}")
    print("\n🎉 Database setup is working perfectly!")
    
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    print("💡 Make sure PostgreSQL is running and your .env file is configured correctly")

# 🗄️ SQLAlchemy Models: Database Structure

Now let's create our database models! These define the actual structure of our database tables.

## Our Todo Model 📋

Let's examine our todo model:

In [ ]:
# Let's look at our models.py file
!cat models.py

## Understanding Our Todo Model 🔍

Let's break down every part of our `Todo` model:

### 1. **Imports**
```python
from sqlalchemy import Boolean, Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship
from database import Base
```
- **SQLAlchemy types**: Define what kind of data each column stores
- **Base**: The parent class from our database.py file

### 2. **Class Definition**
```python
class ToDo(Base):
    __tablename__ = "todos"
```
- **Inherits from Base**: Gets SQLAlchemy superpowers
- **`__tablename__`**: The actual table name in PostgreSQL will be "todos"

### 3. **Column Definitions**
```python
id = Column(Integer, primary_key=True, index=True)
name = Column(String)
completed = Column(Boolean, default=False)
```

Let's understand each column:

#### **id Column**:
- **`Integer`**: Whole numbers (1, 2, 3...)
- **`primary_key=True`**: Unique identifier for each todo
- **`index=True`**: Creates database index for fast lookups

#### **name Column**:
- **`String`**: Text data ("Buy groceries", "Walk the dog")
- **No constraints**: Can be any text (we'll add validation later)

#### **completed Column**:
- **`Boolean`**: True or False values
- **`default=False`**: New todos start as incomplete

## What This Creates in PostgreSQL 🐘

Our model will create this table structure:

```sql
CREATE TABLE todos (
    id SERIAL PRIMARY KEY,      -- Auto-incrementing ID
    name TEXT,                  -- Todo description
    completed BOOLEAN DEFAULT FALSE  -- Completion status
);
```

## Testing Our Model 🧪

In [ ]:
# Test our Todo model
try:
    from models import ToDo
    from database import Base
    
    print("🗄️ Todo Model Test:")
    
    # Check model attributes
    print(f"✅ Model class: {ToDo}")
    print(f"✅ Table name: {ToDo.__tablename__}")
    print(f"✅ Inherits from Base: {issubclass(ToDo, Base)}")
    
    # Check columns
    columns = ToDo.__table__.columns
    print(f"✅ Number of columns: {len(columns)}")
    
    for column in columns:
        print(f"   📋 {column.name}: {column.type} {'(Primary Key)' if column.primary_key else ''}")
    
    # Test creating a model instance (not saved to DB yet)
    test_todo = ToDo(name="Test todo", completed=False)
    print(f"\n🧪 Test instance created:")
    print(f"   Name: {test_todo.name}")
    print(f"   Completed: {test_todo.completed}")
    print(f"   ID: {test_todo.id} (will be assigned when saved to DB)")
    
    print("\n🎉 Todo model is working correctly!")
    
except Exception as e:
    print(f"❌ Model test failed: {e}")

## Advanced SQLAlchemy Features 🚀

Let's explore some powerful SQLAlchemy features we could add to our model:

### Enhanced Model Example

In [ ]:
# Example of a more advanced Todo model (for learning purposes)
from sqlalchemy import Boolean, Column, Integer, String, DateTime, Text
from sqlalchemy.sql import func
from database import Base

class EnhancedTodo(Base):
    __tablename__ = "enhanced_todos"
    
    # Primary key with auto-increment
    id = Column(Integer, primary_key=True, index=True)
    
    # Required name field with length constraint
    name = Column(String(255), nullable=False)  # Max 255 characters, required
    
    # Optional description field
    description = Column(Text, nullable=True)  # Long text, optional
    
    # Completion status
    completed = Column(Boolean, default=False, nullable=False)
    
    # Automatic timestamps
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())
    
    # Priority level (1=low, 2=medium, 3=high)
    priority = Column(Integer, default=1)
    
    def __repr__(self):
        """String representation of the Todo object"""
        return f"<Todo(id={self.id}, name='{self.name}', completed={self.completed})>"

print("🚀 Enhanced Todo model example created!")
print("This shows advanced features like:")
print("  • String length constraints")
print("  • Nullable/non-nullable fields")
print("  • Automatic timestamps")
print("  • Custom string representation")
print("\n💡 For our todo app, the simple model is perfect!")

# 📋 Pydantic Schemas: API Validation

Now let's create our Pydantic schemas! These define how data should look when it travels through our API.

## Our Schema Files 📄

Let's examine our schemas:

In [ ]:
# Let's look at our schemas.py file
!cat schemas.py

## Understanding Our Schemas 🔍

Let's break down each schema:

### 1. **Import**
```python
from pydantic import BaseModel
```
`BaseModel` is the parent class for all Pydantic models.

### 2. **TodoRequest Schema**
```python
class ToDoRequest(BaseModel):
    name: str
    completed: bool
```

**Purpose**: Validates data **coming into** our API
- **When used**: POST requests to create todos, PUT requests to update todos
- **Fields**: Only the data the client needs to provide
- **No ID**: Database generates the ID automatically

### 3. **TodoResponse Schema**
```python
class ToDoResponse(BaseModel):
    name: str
    completed: bool
    id: int

    class Config:
        orm_mode = True
```

**Purpose**: Validates data **going out** from our API
- **When used**: GET requests that return todo data
- **Fields**: All data including database-generated ID
- **`orm_mode = True`**: Allows conversion from SQLAlchemy objects

## The Data Flow Journey 🔄

Let's see how these schemas work in practice:

### **Creating a Todo (POST /todos)**:
```
1. 📱 Client sends: {"name": "Buy milk", "completed": false}
2. 🛡️ TodoRequest validates: ✅ Valid format
3. 🏗️ FastAPI converts to: Todo(name="Buy milk", completed=false)
4. 🗄️ Database saves and assigns: ID = 123
5. 📤 TodoResponse formats: {"id": 123, "name": "Buy milk", "completed": false}
6. 📱 Client receives: Complete todo object with ID
```

### **Getting Todos (GET /todos)**:
```
1. 📱 Client requests: GET /todos
2. 🗄️ Database returns: [Todo(id=123, name="Buy milk", completed=false)]
3. 📤 TodoResponse converts: [{"id": 123, "name": "Buy milk", "completed": false}]
4. 📱 Client receives: JSON array of todos
```

## Testing Our Schemas 🧪

In [ ]:
# Test our Pydantic schemas
try:
    from schemas import ToDoRequest, ToDoResponse
    import json
    
    print("📋 Schema Validation Tests:")
    
    # Test TodoRequest (incoming data)
    print("\n🔽 Testing TodoRequest (incoming data):")
    
    # Valid request data
    valid_request_data = {"name": "Buy groceries", "completed": False}
    todo_request = ToDoRequest(**valid_request_data)
    
    print(f"✅ Valid request created:")
    print(f"   Name: {todo_request.name}")
    print(f"   Completed: {todo_request.completed}")
    print(f"   JSON: {todo_request.json()}")
    
    # Test TodoResponse (outgoing data)
    print("\n🔼 Testing TodoResponse (outgoing data):")
    
    # Valid response data (includes ID)
    valid_response_data = {"id": 1, "name": "Buy groceries", "completed": False}
    todo_response = ToDoResponse(**valid_response_data)
    
    print(f"✅ Valid response created:")
    print(f"   ID: {todo_response.id}")
    print(f"   Name: {todo_response.name}")
    print(f"   Completed: {todo_response.completed}")
    print(f"   JSON: {todo_response.json()}")
    
    print("\n🎉 All schema tests passed!")
    
except Exception as e:
    print(f"❌ Schema test failed: {e}")

## Schema Validation in Action 🛡️

Let's see how Pydantic protects us from bad data:

In [ ]:
# Test schema validation with invalid data
from schemas import ToDoRequest, ToDoResponse

print("🛡️ Schema Validation Tests:")

# Test 1: Missing required field
try:
    invalid_data = {"name": "Missing completed field"}
    TodoRequest(**invalid_data)
    print("❌ This shouldn't work - missing required field!")
except Exception as e:
    print(f"✅ Correctly caught missing field: {str(e)[:60]}...")

# Test 2: Wrong data type
try:
    invalid_data = {"name": "Valid name", "completed": "not-a-boolean"}
    ToDoRequest(**invalid_data)
    print("❌ This shouldn't work - wrong data type!")
except Exception as e:
    print(f"✅ Correctly caught type error: {str(e)[:60]}...")

# Test 3: Extra unwanted fields
try:
    data_with_extra = {
        "name": "Valid name", 
        "completed": True,
        "hacker_field": "malicious data",
        "another_extra": 12345
    }
    clean_request = ToDoRequest(**data_with_extra)
    print(f"✅ Extra fields ignored - only got: {clean_request.dict()}")
except Exception as e:
    print(f"Error with extra fields: {e}")

print("\n💡 Pydantic automatically validates and cleans our data!")

## Understanding orm_mode 🔄

The `orm_mode = True` setting is crucial for converting between SQLAlchemy objects and Pydantic schemas.

### Without orm_mode ❌
```python
# This would fail!
db_todo = Todo(id=1, name="Buy milk", completed=False)  # SQLAlchemy object
response = TodoResponse(db_todo)  # ❌ Error - can't convert directly
```

### With orm_mode ✅
```python
# This works perfectly!
db_todo = Todo(id=1, name="Buy milk", completed=False)  # SQLAlchemy object
response = TodoResponse.from_orm(db_todo)  # ✅ Automatic conversion!
```

Let's test this:

In [ ]:
# Test ORM mode conversion
from models import ToDo
from schemas import ToDoResponse

print("🔄 ORM Mode Conversion Test:")

# Create a SQLAlchemy model instance
db_todo = ToDo(id=42, name="Test todo from database", completed=True)
print(f"📋 Database object: {db_todo}")
print(f"   Type: {type(db_todo)}")
print(f"   ID: {db_todo.id}")
print(f"   Name: {db_todo.name}")
print(f"   Completed: {db_todo.completed}")

# Convert to Pydantic schema using ORM mode
try:
    response_schema = ToDoResponse.from_orm(db_todo)
    print(f"\n✅ Converted to API response:")
    print(f"   Type: {type(response_schema)}")
    print(f"   JSON: {response_schema.json()}")
    
    # This is what gets sent to the client
    print(f"\n📤 Client receives: {response_schema.dict()}")
    
except Exception as e:
    print(f"❌ Conversion failed: {e}")

print("\n🎯 This is how we convert database objects to API responses!")

# 🎨 Schema Design Patterns

Let's explore some common patterns for designing schemas in real applications.

## Different Schemas for Different Operations 🔄

In larger applications, you might have multiple schemas for the same model:

In [ ]:
# Example of multiple schemas for different use cases
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

# Base schema with common fields
class TodoBase(BaseModel):
    name: str = Field(..., min_length=1, max_length=255, description="Todo description")
    completed: bool = Field(default=False, description="Completion status")

# For creating new todos
class TodoCreate(TodoBase):
    pass  # Same as base - no ID needed

# For updating existing todos (all fields optional)
class TodoUpdate(BaseModel):
    name: Optional[str] = Field(None, min_length=1, max_length=255)
    completed: Optional[bool] = None

# For API responses (includes all fields)
class TodoResponse(TodoBase):
    id: int = Field(..., description="Unique identifier")
    created_at: Optional[datetime] = None
    updated_at: Optional[datetime] = None
    
    class Config:
        orm_mode = True
        schema_extra = {
            "example": {
                "id": 1,
                "name": "Buy groceries",
                "completed": False,
                "created_at": "2023-12-01T10:00:00Z"
            }
        }

# For listing todos (maybe without timestamps)
class TodoSummary(BaseModel):
    id: int
    name: str
    completed: bool
    
    class Config:
        orm_mode = True

print("🎨 Advanced schema patterns created!")
print("Different schemas for different operations:")
print("  📝 TodoCreate - for POST requests")
print("  ✏️ TodoUpdate - for PATCH requests (partial updates)")
print("  📤 TodoResponse - for detailed GET responses")
print("  📋 TodoSummary - for list endpoints")
print("\n💡 For our simple todo app, our current schemas are perfect!")

## Pydantic Validation Features 🛡️

Let's explore some powerful validation features:

In [ ]:
# Example of advanced validation
from pydantic import BaseModel, Field, validator
from typing import Optional

class ValidatedTodoRequest(BaseModel):
    name: str = Field(
        ...,  # Required field
        min_length=1,
        max_length=100,
        description="Todo name (1-100 characters)"
    )
    completed: bool = Field(default=False, description="Completion status")
    priority: Optional[int] = Field(default=1, ge=1, le=5, description="Priority 1-5")
    
    @validator('name')
    def validate_name(cls, v):
        # Remove extra whitespace
        v = v.strip()
        
        # Check for empty string after stripping
        if not v:
            raise ValueError('Name cannot be empty or just whitespace')
        
        # Check for inappropriate content
        forbidden_words = ['hack', 'virus', 'malware']
        if any(word in v.lower() for word in forbidden_words):
            raise ValueError('Name contains inappropriate content')
            
        return v
    
    @validator('priority')
    def validate_priority(cls, v):
        if v is not None and (v < 1 or v > 5):
            raise ValueError('Priority must be between 1 and 5')
        return v

# Test the validation
print("🛡️ Testing Advanced Validation:")

# Test valid data
try:
    valid_todo = ValidatedTodoRequest(name="  Buy milk  ", priority=3)
    print(f"✅ Valid todo: {valid_todo.dict()}")
    print(f"   Notice: name was trimmed from '  Buy milk  ' to '{valid_todo.name}'")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

# Test invalid data
try:
    invalid_todo = ValidatedTodoRequest(name="hack the system", priority=10)
    print("❌ This should have been rejected!")
except Exception as e:
    print(f"✅ Correctly rejected: {e}")

print("\n💡 Pydantic provides powerful validation capabilities!")

# 🤝 Models and Schemas Working Together

Now let's see how our models and schemas work together in practice.

## The Complete Data Journey 🔄

Let's simulate the complete flow from API request to database and back:

In [ ]:
# Simulate the complete data flow
from models import ToDo
from schemas import ToDoRequest, ToDoResponse

print("🔄 Complete Data Flow Simulation:")

# Step 1: Simulate incoming API request
print("\n📥 Step 1: Client sends API request")
incoming_json = '{"name": "Buy groceries", "completed": false}'
print(f"   Raw JSON: {incoming_json}")

# Step 2: Pydantic validates the request
print("\n🛡️ Step 2: Pydantic validates request")
try:
    import json
    raw_data = json.loads(incoming_json)
    validated_request = ToDoRequest(**raw_data)
    print(f"   ✅ Validation successful: {validated_request.dict()}")
except Exception as e:
    print(f"   ❌ Validation failed: {e}")
    exit()

# Step 3: Convert to database model
print("\n🗄️ Step 3: Create database model")
db_todo = ToDo(**validated_request.dict())
# Simulate database assigning an ID
db_todo.id = 42  # In real app, database would assign this
print(f"   📋 Database object created: id={db_todo.id}, name='{db_todo.name}', completed={db_todo.completed}")

# Step 4: Convert back to API response
print("\n📤 Step 4: Create API response")
api_response = ToDoResponse.from_orm(db_todo)
print(f"   📋 Response object: {api_response.dict()}")
print(f"   📋 Response JSON: {api_response.json()}")

print("\n🎉 Complete flow successful!")
print("\n📊 Summary of transformations:")
print(f"   📥 JSON string → ToDoRequest → ToDo model → ToDoResponse → JSON string 📤")
print(f"   🔄 Each step validates and transforms the data safely!")

## Common Integration Patterns 🔧

Here are the patterns you'll use in your FastAPI endpoints:

In [ ]:
# Common patterns you'll use in FastAPI endpoints
from models import ToDo
from schemas import ToDoRequest, ToDoResponse
from typing import List

print("🔧 Common Integration Patterns:")

# Pattern 1: Create (POST endpoint)
def create_todo_pattern(todo_data: ToDoRequest) -> ToDoResponse:
    """Pattern for creating a new todo"""
    # Convert request to database model
    db_todo = ToDo(**todo_data.dict())
    
    # In real app: save to database
    # db.add(db_todo)
    # db.commit()
    # db.refresh(db_todo)
    
    # Simulate database assigning ID
    db_todo.id = 1
    
    # Convert to response
    return ToDoResponse.from_orm(db_todo)

# Pattern 2: Read (GET endpoint)
def get_todos_pattern(db_todos: List[ToDo]) -> List[ToDoResponse]:
    """Pattern for getting multiple todos"""
    # Convert list of database models to list of responses
    return [ToDoResponse.from_orm(todo) for todo in db_todos]

# Pattern 3: Update (PUT endpoint)
def update_todo_pattern(todo_id: int, todo_data: ToDoRequest) -> ToDoResponse:
    """Pattern for updating a todo"""
    # In real app: get from database
    # db_todo = db.query(ToDo).filter(ToDo.id == todo_id).first()
    
    # Simulate existing todo
    db_todo = ToDo(id=todo_id, name="Old name", completed=False)
    
    # Update fields
    db_todo.name = todo_data.name
    db_todo.completed = todo_data.completed
    
    # In real app: save changes
    # db.commit()
    # db.refresh(db_todo)
    
    # Return updated todo
    return ToDoResponse.from_orm(db_todo)

# Test the patterns
print("\n🧪 Testing patterns:")

# Test create pattern
request_data = ToDoRequest(name="Test todo", completed=False)
created_todo = create_todo_pattern(request_data)
print(f"✅ Create pattern: {created_todo.dict()}")

# Test read pattern
mock_db_todos = [
    ToDo(id=1, name="Todo 1", completed=False),
    ToDo(id=2, name="Todo 2", completed=True)
]
todo_list = get_todos_pattern(mock_db_todos)
print(f"✅ Read pattern: {len(todo_list)} todos")

# Test update pattern
update_data = ToDoRequest(name="Updated todo", completed=True)
updated_todo = update_todo_pattern(1, update_data)
print(f"✅ Update pattern: {updated_todo.dict()}")

print("\n🎯 These patterns will be used in our actual API endpoints!")

# ✅ Models and Schemas Status Check

Let's verify that everything is working correctly.

## Model Verification 🗄️

In [ ]:
# Comprehensive model verification
print("🗄️ Model Verification:")

try:
    from models import ToDo
    from database import Base, engine
    
    # Check model inheritance
    print(f"✅ ToDo inherits from Base: {issubclass(ToDo, Base)}")
    
    # Check table name
    print(f"✅ Table name: {ToDo.__tablename__}")
    
    # Check columns
    table = ToDo.__table__
    print(f"✅ Number of columns: {len(table.columns)}")
    
    column_info = []
    for column in table.columns:
        info = f"{column.name}({column.type})"
        if column.primary_key:
            info += " PK"
        if column.default is not None:
            info += f" default={column.default.arg}"
        column_info.append(info)
    
    print(f"✅ Columns: {', '.join(column_info)}")
    
    # Test model creation
    test_todo = ToDo(name="Test", completed=False)
    print(f"✅ Model instance created: {test_todo.name}, {test_todo.completed}")
    
    print("\n🎉 Model verification successful!")
    
except Exception as e:
    print(f"❌ Model verification failed: {e}")

## Schema Verification 📋

In [ ]:
# Comprehensive schema verification
print("📋 Schema Verification:")

try:
    from schemas import ToDoRequest, ToDoResponse
    from pydantic import BaseModel
    
    # Check inheritance
    print(f"✅ ToDoRequest inherits from BaseModel: {issubclass(ToDoRequest, BaseModel)}")
    print(f"✅ ToDoResponse inherits from BaseModel: {issubclass(ToDoResponse, BaseModel)}")
    
    # Check fields
    request_fields = list(ToDoRequest.__fields__.keys())
    response_fields = list(ToDoResponse.__fields__.keys())
    
    print(f"✅ ToDoRequest fields: {request_fields}")
    print(f"✅ ToDoResponse fields: {response_fields}")
    
    # Check ORM mode
    has_orm_mode = hasattr(ToDoResponse.Config, 'orm_mode') and ToDoResponse.Config.orm_mode
    print(f"✅ ToDoResponse has orm_mode: {has_orm_mode}")
    
    # Test schema creation
    test_request = ToDoRequest(name="Test", completed=False)
    test_response = ToDoResponse(id=1, name="Test", completed=False)
    
    print(f"✅ Request schema created: {test_request.dict()}")
    print(f"✅ Response schema created: {test_response.dict()}")
    
    # Test JSON serialization
    print(f"✅ Request JSON: {test_request.json()}")
    print(f"✅ Response JSON: {test_response.json()}")
    
    print("\n🎉 Schema verification successful!")
    
except Exception as e:
    print(f"❌ Schema verification failed: {e}")

## Integration Verification 🤝

In [ ]:
# Test integration between models and schemas
print("🤝 Integration Verification:")

try:
    from models import ToDo
    from schemas import ToDoRequest, ToDoResponse
    
    # Test: Request → Model → Response flow
    print("\n🔄 Testing complete integration flow:")
    
    # Step 1: Create request
    request = ToDoRequest(name="Integration test", completed=False)
    print(f"1️⃣ Request created: {request.dict()}")
    
    # Step 2: Convert to model
    model = ToDo(**request.dict())
    model.id = 99  # Simulate database ID assignment
    print(f"2️⃣ Model created: id={model.id}, name='{model.name}', completed={model.completed}")
    
    # Step 3: Convert to response
    response = ToDoResponse.from_orm(model)
    print(f"3️⃣ Response created: {response.dict()}")
    
    # Verify data integrity
    assert request.name == response.name
    assert request.completed == response.completed
    assert response.id == 99
    
    print("\n✅ Data integrity verified through complete flow!")
    
    # Test field type consistency
    print("\n🔍 Field type verification:")
    print(f"   Request name type: {type(request.name)}")
    print(f"   Model name type: {type(model.name)}")
    print(f"   Response name type: {type(response.name)}")
    
    print(f"   Request completed type: {type(request.completed)}")
    print(f"   Model completed type: {type(model.completed)}")
    print(f"   Response completed type: {type(response.completed)}")
    
    print(f"   Response id type: {type(response.id)}")
    
    print("\n🎉 Integration verification successful!")
    
except Exception as e:
    print(f"❌ Integration verification failed: {e}")
    import traceback
    traceback.print_exc()

# 🎉 Models and Schemas Complete!

Outstanding work! You've successfully created a robust data layer for your todo application.

## 🏆 What You've Accomplished:

### **Database Layer (SQLAlchemy Models)**:
- ✅ **ToDo model** defined with proper columns and types
- ✅ **Database connection** setup with engine and session factory
- ✅ **Base class** ready for table creation
- ✅ **Primary key** and default values configured

### **API Layer (Pydantic Schemas)**:
- ✅ **ToDoRequest** schema for incoming data validation
- ✅ **ToDoResponse** schema for outgoing data formatting
- ✅ **ORM mode** enabled for seamless model conversion
- ✅ **Type safety** and validation throughout

### **Integration Layer**:
- ✅ **Seamless conversion** between models and schemas
- ✅ **Data integrity** maintained through complete flow
- ✅ **Professional patterns** for CRUD operations
- ✅ **Error handling** and validation at every step

## 📚 Key Concepts You've Mastered:

- **🗄️ SQLAlchemy Models**: How to define database table structure
- **📋 Pydantic Schemas**: How to validate and structure API data
- **🔄 ORM Mode**: Converting between database objects and API responses
- **🛡️ Data Validation**: Protecting your API from invalid data
- **🎯 Separation of Concerns**: Database layer vs API layer responsibilities

## 🚀 What's Next:

In the next notebook ("**Database Migrations with Alembic**"), you'll learn:
- How to create and manage database schema changes
- Setting up Alembic for database migrations
- Creating the actual todos table in PostgreSQL
- Understanding migration versioning and rollbacks

## 💡 Quick Reference:

**Your data models are now ready:**
```python
# Database model (models.py)
class ToDo(Base):
    __tablename__ = "todos"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String)
    completed = Column(Boolean, default=False)

# API schemas (schemas.py)
class ToDoRequest(BaseModel):
    name: str
    completed: bool

class ToDoResponse(BaseModel):
    id: int
    name: str
    completed: bool
    class Config:
        orm_mode = True
```

**Integration pattern:**
```python
# Request → Model → Response
request = ToDoRequest(name="Buy milk", completed=False)
model = ToDo(**request.dict())
response = ToDoResponse.from_orm(model)
```

Your data layer is now bulletproof and ready for database operations! 🎯

**Excellent work on mastering data modeling! You're building like a professional developer! 🌟**